<a href="https://colab.research.google.com/github/k-walsh/tunes-over-time/blob/main/spotify_data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import sqlite3
import csv
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#from bs4 import BeautifulSoup
import requests
import json 
import pprint
!pip install unidecode
import unidecode
from difflib import SequenceMatcher

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

conn = sqlite3.connect('billboard.db') # automatically creates new file for 
c = conn.cursor()
c.execute('PRAGMA foreign_keys = ON')

     |████████████████████████████████| 245kB 6.8MB/s 


In [ ]:
c.execute('''DROP TABLE IF EXISTS billboard;''')

create_billboard = '''
CREATE TABLE billboard(
  "date" TEXT,
  "rank" INT,
  "song" TEXT,
  "artist" TEXT,
  "last_week" INT,
  "peak_rank" INT,
  "weeks_on_board" INT
);
'''

# Use the execute function to execute the command
c.execute(create_billboard)

# And then commit the changes to the file
conn.commit()

a_file = open("/content/drive/MyDrive/charts.csv")
rows = csv.reader(a_file)
c.executemany("INSERT INTO billboard VALUES (?, ?, ?, ?, ?, ?, ?)", rows)
conn.commit()

In [ ]:
c.execute('''
    UPDATE billboard
    SET date = substr(date, 0, 5);''')

conn.commit()

In [ ]:
c.execute('''DROP TABLE IF EXISTS billboards_filtered;''')

c.execute('''
    CREATE TABLE billboards_filtered AS
        WITH summary AS (
            SELECT b.date, 
                   b.song, 
                   b.artist,
                   b.peak_rank,
                   b.weeks_on_board,
                   ROW_NUMBER() OVER(PARTITION BY b.song, b.artist 
                                         ORDER BY b.weeks_on_board DESC) AS rank
              FROM billboard b)
         SELECT *
           FROM summary
         WHERE rank = 1;''')

conn.commit()

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 24))



OperationalError: ignored

In [ ]:
c.execute('''DROP TABLE IF EXISTS popular_songs;''')

c.execute('''
CREATE TABLE popular_songs(
  "date" TEXT,
  "song" TEXT,
  "artist" TEXT,
  "peak_rank" TEXT,
  "weeks_on_board" TEXT
);''')

for yr in range(1958, 2021):
    c.execute('''
    INSERT INTO popular_songs (date, song, artist, peak_rank, weeks_on_board) 
    SELECT date as year, song, artist, peak_rank, weeks_on_board
    FROM billboards_filtered
    where date = ''' + str(yr) + ''' 
    order by peak_rank asc, weeks_on_board desc
    limit 75;''')
    
conn.commit()

In [ ]:
c.execute('''DROP TABLE billboard;''')
c.execute('''DROP TABLE billboards_filtered;''')

In [ ]:
c.execute('''
SELECT song, artist
FROM popular_songs;''')

songs_list = c.fetchall()

In [ ]:
############# SPOTIFY #############

In [ ]:
CLIENT_ID = '80a481dc1cee412f91416881e7a11029'
CLIENT_SECRET = '51addb341d9a41ba8778f460114a880e'

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

In [ ]:
def replace_special_char(name):
    name = name.replace(" & ", " ")
    name = name.replace(" - ", " ")
    name = name.replace(" / ", " ")
    name = name.replace(" + ", " ")
    name = name.replace("&", " ")
    name = name.replace("-", " ")
    name = name.replace("/", " ")
    name = name.replace("+", " ")
    name = name.replace(" and ", " ")
    return name


def clean_song(song):
    song = song.lower()
    split_on_spaces = song.split(' ')
    name = ''
    
    for word in split_on_spaces:
        if 'feat.' in word or 'featuring' in word:
            break
        name += word + " "
    
    name = name.strip()
    unaccented_string = unidecode.unidecode(name) 
    
    return replace_special_char(unaccented_string)


def clean_artist(artist):
    artist = artist.lower()
    
    if "featuring" in artist:
        artist = artist.replace("featuring","")
    if "feat." in artist:
        artist = artist.replace("feat.","")
    
    unaccented_string = unidecode.unidecode(artist)
    
    return replace_special_char(unaccented_string)

In [ ]:
def request_song(song, artist):
    query = requests.utils.quote(song + " " + artist)
    r = requests.get(BASE_URL + 'search/?q=' + query + '&type=track', headers=headers)
    #print(BASE_URL + 'search/?q=' + query + '&type=track')
    print(r.status_code)
    return r.json()

In [ ]:
c.execute('''
ALTER TABLE popular_songs
ADD spotify_id text;''')

c.execute('''
ALTER TABLE popular_songs
ADD spotify_song_name text;''')

c.execute('''
ALTER TABLE popular_songs
ADD spotify_song_artist text;''')

In [ ]:
c.execute('''
    UPDATE popular_songs
    SET song = quote(song), artist = quote(artist);''')

In [ ]:
def get_song_ids(x1, x2):
    list_ids = []
    
    for (given_song_org, given_artist_org) in songs_list[x1:x2]:

        song_id = None
        cleaned_song = ''

        given_song = clean_song(given_song_org.lower())
        given_artist = clean_artist(given_artist_org.lower())

        r = request_song(given_song, given_artist)

        max_sim_ratio_song = 0
        max_sim_ratio_artist = 0
        max_sim_ratio = 0

        #print('--------')
        #print('given:', ' song:', given_song_org, ' artist:', given_artist_org)

        for track in r['tracks']['items']:

            spotify_song_org = track['name']
            spotify_song = clean_song(spotify_song_org)
            all_spotify_artists = ''

            for artist in track['artists']:      

                spotify_artist = clean_artist(artist['name'])
                all_spotify_artists += " " + spotify_artist
                
            all_spotify_artists = all_spotify_artists.strip()

            sim_ratio_song = SequenceMatcher(None, given_song, spotify_song).ratio()
            sim_ratio_artist = SequenceMatcher(None, given_artist, all_spotify_artists).ratio()

            #sim_ratio = sim_ratio_song * 0.4 + sim_ratio_artist * 0.6
            sim_ratio = sim_ratio_song + sim_ratio_artist

            if sim_ratio_song >= 0.6 and sim_ratio_artist >= 0.6 and sim_ratio > max_sim_ratio:
                max_sim_ratio_song = sim_ratio_song
                max_sim_ratio_artist = sim_ratio_artist
                max_sim_ratio = sim_ratio
                song_id = track['id']

            #print('spotify:', ' song:', spotify_song, ' artists:', all_spotify_artists, 'song: ', sim_ratio_song, 'artist:', sim_ratio_artist, ' sim ratio =', sim_ratio)  

        if song_id == None:
            print('NOT FOUND')
        else: print('id:', song_id)
        print('--------')

        # going to replace ones with none, manually
        if song_id == None: 
            song_id = 'None'

        list_ids.append((song_id, given_song_org, given_artist_org))
#         c.execute('''
#         UPDATE popular_songs
#         SET spotify_id = "''' + song_id + '''", spotify_song_name = quote("''' 
#                   + spotify_song_org + '''"), spotify_song_artist = quote("''' 
#                   + all_spotify_artists + '''")
#         WHERE song = quote("''' + given_song_org + '") and artist = quote("' + given_artist_org + '");')

        query = '''
        UPDATE popular_songs
        SET spotify_id = ?, spotify_song_name = ?, spotify_song_artist = ?
        WHERE song = ? and artist = ?;'''
    
        tuple1 = (song_id, spotify_song_org, all_spotify_artists, given_song_org, given_artist_org)
    
        c.execute(query, tuple1)
        
    conn.commit()
    return list_ids

    #     c.execute('''
    #     INSERT INTO popular_songs (spotify_song_name) 
    #     VALUES ("''' + spotify_song_org.replace('"', '') + '");')   

    #     c.execute('''
    #     INSERT INTO popular_songs (spotify_song_artist) 
    #     VALUES ("''' + all_spotify_artists.replace('"', '') + '");')    

    
    
  
     #  just to see the songs that aren't found
    #     if song_id == None:
    #         print('NOT FOUND')
    #         print('given: song:', given_song, ' artists:', given_artist)
    #         print('///')

In [ ]:
first_1000 = get_song_ids(0, 1000)

200
id: 3c7KT5CN8uYRaK3xThhdYt
--------
200
id: 0SECl46T8opPOK8pOTHq28
--------
200
id: 0p1c2oNEQDd9MF1H3R9p3z
--------
200
NOT FOUND
--------
200
id: 2ZCkqAo0tzzCVOth7ityh5
--------
200
id: 205EtD8tQnHDXMkmtYiaHF
--------
200
id: 2Un1O9CChp4wyTmt422XwW
--------
200
id: 40fD7ct05FvQHLdQTgJelG
--------
200
id: 07ePHsQ0SopWrnCbNjNC2D
--------
200
id: 1fDKsQrMYOTGHYgyIxycmM
--------


In [ ]:
to_2000 = get_song_ids(1000, 2000)

In [ ]:
to_3000 = get_song_ids(2000, 3000)

In [ ]:
to_4000 = get_song_ids(3000, 4000)

In [ ]:
to_end = get_song_ids(4000, len(songs_list) + 1)

In [ ]:
# c.execute('''SELECT * FROM popular_songs LIMIT 100;''')
# c.fetchall()

[('1958',
  "'Little Star'",
  "'The Elegants'",
  '1',
  '17',
  '3c7KT5CN8uYRaK3xThhdYt',
  "'Twinkle Twinkle Little Star'",
  "' the elegants'"),
 ('1958',
  "'Nel Blu Dipinto Di Blu (Volaré)'",
  "'Domenico Modugno'",
  '1',
  '16',
  '0SECl46T8opPOK8pOTHq28',
  "'Nel Blu Dipinto Di Blu (Volare)'",
  "' domenico modugno'"),
 ('1958',
  "'Poor Little Fool'",
  "'Ricky Nelson'",
  '1',
  '11',
  '0p1c2oNEQDd9MF1H3R9p3z',
  "'Poor Little Fool (In The Style Of Ricky Nelson)'",
  "' ameritz audio karaoke'"),
 ('1958',
  "'Rock-in Robin'",
  "'Bobby Day'",
  '2',
  '21',
  'None',
  "'Rockin'' Robin (Karaoke Lead Vocal Demo)[In the style of Bobby Day]'",
  "' prosound karaoke band'"),
 ('1958',
  "'Bird Dog'",
  "'The Everly Brothers'",
  '2',
  '18',
  '2ZCkqAo0tzzCVOth7ityh5',
  "'All I Have to Do Is Dream (Live)'",
  "' the wieners'"),
 ('1958',
  "'Patricia'",
  "'Perez Prado And His Orchestra'",
  '2',
  '14',
  'None',
  "'Patricia'",
  "' perez prado'"),
 ('1958',
  "'My True Love

In [ ]:
# code used to convert the popular songs table in the db to a csv

# df = pd.read_sql_query("SELECT * FROM popular_songs", conn)

# df.to_csv ('/Users/kiera/Downloads/popular_songs.csv', index = False)